In [1]:
import sqlalchemy as sa
import urllib
import pandas as pd
import pyodbc
import time
import glob
from datetime import datetime, timedelta
from os import listdir
from os.path import isfile, join
from  tqdm import tqdm
pd.set_option('display.max_columns', 41)


In [2]:
#Connect databse 34
connection_string = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'SERVER=118.69.201.34;'
    'Database=FLC_SHOP_SALES_DATAWAREHOUSE;'
    'UID=ecom_user;'
    'PWD=Ec0m@12345;'
    'Trusted_Connection=no;'
)

connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(str(connection_string))}"
engine = sa.create_engine(connection_uri, fast_executemany=True)

In [ ]:
def getEngine(engine_name):
    connection = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(engine_name)}"
    engine = sa.create_engine(connection, fast_executemany=True)
    return engine
# Dựa vào engine_name nhận được, hàm sử dụng module urllib.parse để mã hóa engine_name 
# thành một chuỗi URL an toàn, sử dụng hàm quote_plus. Chuỗi URL này sẽ được sử dụng để tạo kết nối với cơ sở dữ liệu.
# Tiếp theo, hàm tạo một chuỗi kết nối (connection) bằng cách ghép nối chuỗi "mssql+pyodbc:///?odbc_connect=" 
# với chuỗi đã được mã hóa từ engine_name.
# Sử dụng connection đã tạo, hàm sử dụng hàm create_engine từ SQLAlchemy để tạo một đối tượng Engine. 
# Đối tượng Engine này sẽ đại diện cho kết nối tới cơ sở dữ liệu MSSQL.
# Cuối cùng, hàm trả về đối tượng Engine đã được tạo.
# Điểm đáng chú ý là trong câu lệnh tạo đối tượng Engine, tùy chọn fast_executemany=True được cung cấp.
# Tùy chọn này giúp cải thiện hiệu suất khi thực thi các truy vấn lặp lại trong SQLAlchemy.
# Tóm lại, hàm getEngine được sử dụng để tạo và trả về một đối tượng Engine để kết nối đến cơ sở dữ liệu MSSQL,
# dựa trên thông tin engine_name đã được truyền vào.

In [ ]:

### Connect to DataMart Server
mart_server = '118.69.201.34'
mart_database = 'FLC_SHOP_SALES_DATAWAREHOUSE'
mart_username = 'ecom_user'
mart_password = 'Ec0m@12345'
SaleMart = ('DRIVER={ODBC Driver 17 for SQL Server}'
            ';SERVER=' + mart_server +
            ';DATABASE=' + mart_database +
            ';UID=' + mart_username +
            ';PWD=' + mart_password)

In [3]:
### Connect to Azure server
azr_server = "ssd-synapse-prod.sql.azuresynapse.net"
azr_database = 'frtssdsqlpoolprod'
azr_username = 'frtcid'
azr_password = 'Frt2022Cid@#123'       
connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username=azr_username,
    password=azr_password,
    host=azr_server,
    database=azr_database,
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "autocommit": "True",
    },
)
azr_engine = sa.create_engine(connection_url)

In [5]:
sales_trans_flc = f"""SELECT DATE_KEY, COUNT(*) COUNT_LINE, 'daitt5' AS INSERT_BY
FROM dbo.VF_CID_SALES_TRANSACTION_FLC
GROUP BY DATE_KEY
ORDER BY DATE_KEY"""
trans_flc = pd.read_sql_query(sales_trans_flc,azr_engine)
sales_trans_ict = f"""SELECT DATE_KEY, COUNT(*) COUNT_LINE, 'daitt5' AS INSERT_BY
FROM dbo.VF_CID_SALES_TRANSACTION_ICT
GROUP BY DATE_KEY
ORDER BY DATE_KEY"""
trans_ict = pd.read_sql_query(sales_trans_ict,azr_engine)

In [6]:
trans_flc

,DATE_KEY,COUNT_LINE,INSERT_BY
0,20190101,12959,daitt5
1,20190102,15480,daitt5
2,20190103,15803,daitt5
3,20190104,14740,daitt5
4,20190105,14325,daitt5
...,...,...,...
1639,20230628,524489,daitt5
1640,20230629,517370,daitt5
1641,20230630,516432,daitt5
1642,20230701,502264,daitt5


In [10]:
# Lấy thời gian hiện tại
current_time = datetime.now()
formatted_time = current_time.strftime("%Y%m%d")
trans_flc['DATE_KEY'] = trans_flc['DATE_KEY'].astype(str)
trans_ict['DATE_KEY'] = trans_ict['DATE_KEY'].astype(str)
#Lấy những record của những ngày nhỏ hơn ngày hiện tại
trans_flc = trans_flc.loc[trans_flc['DATE_KEY'] < formatted_time]
trans_ict = trans_ict.loc[trans_ict['DATE_KEY'] < formatted_time]


In [12]:
trans_flc

,DATE_KEY,COUNT_LINE,INSERT_BY
0,20190101,12959,daitt5
1,20190102,15480,daitt5
2,20190103,15803,daitt5
3,20190104,14740,daitt5
4,20190105,14325,daitt5
...,...,...,...
1639,20230628,524489,daitt5
1640,20230629,517370,daitt5
1641,20230630,516432,daitt5
1642,20230701,502264,daitt5


In [ ]:
trans_flc.to_sql("FOLLOW_TRANSACTION_FLC", engine, schema="dbo", if_exists="append", index=False)
trans_ict.to_sql("FOLLOW_TRANSACTION_ICT", engine, schema="dbo", if_exists="append", index=False)